<h1><center>YanP_02</center></h1>


<h3 align="center">Oct 9, 2022</h3> 

0.1  Student: Yan, Puwentao (V00031753)
## 1 Problem 2

In [25]:
# installing basic python3 library
!pip freeze
!pip3 install numpy
!pip3 install pandas
!pip3 install matplotlib
!pip3 install six

absl-py==1.2.0
aeppl==0.0.33
aesara==2.7.9
aiohttp==3.8.3
aiosignal==1.2.0
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==3.0.0
autograd==1.5
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.8
bokeh==2.3.3
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.8
certifi==2022.9.24
cffi==1.15.1
cftime==1.6.2
chardet==3.0.4
charset-normalizer==2.1.1
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.7
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.2
cons==0.4.5
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.2.1
cycler==0.11.0
cymem==2.0.6
Cython==0.29.32
daft==0.0.4
dask==2022.2.0
datascience==0.17.5
debugpy==1.0.0
decorator==4.4.

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import math
np.random.seed(1337)

1.1 Implement ID3 from scratch

In [27]:
# import elections_clean.csv from google drive
url = 'https://drive.google.com/file/d/1wAKUvFUSTuh-0bdKjzi2rnA-gKXRqPhu/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)

In [28]:
catg_features = ['education', 'religion', 'ethnic_female', 'ethnic_male']
tot_n_row = df.shape[0] # total # of rows
id3_df = df
new_catg_features = catg_features[:]

class Tree(object):
  def __init__(self, feature):
    self.feature = feature
    self.children = []
    self.label = None

  def set_feature(self, feature):
    self.feature = feature

  def add_child(self, child):
    self.children.append(child)
  
  def set_label(self, label):
    self.label = label
  
  def __str__(self, space=0):
    if self.label != None:
      res = ":   {"+repr(self.feature)+" = "+str(self.label)+"}"+"\n"
    else:
      res = "\n"+"|        "*space+repr(self.feature)
    for i in self.children:
        res += i.__str__(space+1)
    return res

def cal_entropy(df, label): # df: dataframe, label: name of the label column of df
  n_row = len(df[label]) # number of rows
  n_0 = list(df[label].values == 0).count(True)
  n_1 = list(df[label].values == 1).count(True)
  if n_0 == 0 or n_1 == 0:
    entropy = 0
  else:
    entropy = -(n_1/n_row)*np.log2(n_1/n_row)-(n_0/n_row)*np.log2(n_0/n_row)
  label_v_num = {0: n_0, 1: n_1} # contains label values and their corrsponding amount
  # entropy_list = [entropy, most represented value]
  entropy_list = [entropy, max(label_v_num, key=label_v_num.get)]
  return entropy_list

def cal_cond_entropy(feature, sample, temp_df):
  
  n_row = list(temp_df[feature].values == sample).count(True)
  n_0 = list(temp_df['democrat'].loc[temp_df[feature].values == sample].values == 0).count(True)
  n_1 = list(temp_df['democrat'].loc[temp_df[feature].values == sample].values == 1).count(True)
  # eliminate the situation of np.log2(0)
  if n_0 == 0 or n_1 == 0:
    entropy = 0
  else:
    entropy = -(n_1/n_row)*np.log2(n_1/n_row)-(n_0/n_row)*np.log2(n_0/n_row)
  return entropy

def get_best_feature(features, temp_df): # features: column names list [str, str]
  features_ig = {} # contains every feature's IG
  total_entropy = cal_entropy(df, 'democrat')[0]

  for feature in features:
    cond_entropy = 0
    
    for sample in temp_df[feature].unique(): # obtain distinct sample names in a feature to calculate its entropy
      pr = (list(temp_df[feature].values == sample).count(True)) / tot_n_row
      # conditional entropy
      cond_entropy += pr*cal_cond_entropy(feature, sample, temp_df)
    features_ig[feature] = total_entropy - cond_entropy
  return max(features_ig, key = features_ig.get)

def id3_entropy(id3_df, catg_features):
  tree = Tree('democrat') # set a tree node with feature = democrat since this node could be a leaf representing democrat value. If it's not a leaf, we reset the feature name in the 'else' branch.
  entropy_list = cal_entropy(id3_df, 'democrat')
  
  if entropy_list[0] == 0: # check if node is pure
    tree.set_label(entropy_list[1])
    return tree
  elif len(catg_features) == 0: # check if categorical features is empty
    tree.set_label(entropy_list[1])
    return tree
  else:
    # obtain best feature and add it to a new node
    best_feature = get_best_feature(catg_features, id3_df)
    tree.set_feature(best_feature)
    # remove the best feature
    catg_features.remove(best_feature)
    i = 0 # index of child node
    for value in list(id3_df[best_feature].unique()):
      # create child node for tree with best feature
      node = Tree(value)
      tree.add_child(node)
      value_df = id3_df.loc[id3_df[best_feature].values == value] # create a dataframe only contain 'value'
      if value_df.shape[0] == 0: # value_df is empty
        leaf = Tree('democrat')
        entropy_list2 = cal_entropy(value_df, 'democrat')
        leaf.set_label(entropy_list2[1])
        tree.children[i].add_child(leaf)
      else:
        new_catg_features = catg_features[:] # obtain the copy of catg_features so that the iteration won't damage the orginal catg_features
        tree.children[i].add_child(id3_entropy(value_df, new_catg_features))
      i += 1
  return tree
id3_tree = id3_entropy(df, new_catg_features)
# print the decision tree!
str(id3_tree)
print(id3_tree)



'ethnic_male'
|        'MULTI MALE'
|        |        'education'
|        |        |        'Bachelor'
|        |        |        |        'religion'
|        |        |        |        |        'CHRISTIAN GENERIC':   {'democrat' = 0}

|        |        |        |        |        'MAINLINE CHRISTIAN'
|        |        |        |        |        |        'ethnic_female'
|        |        |        |        |        |        |        'BLACK FEMALE':   {'democrat' = 0}

|        |        |        |        |        |        |        'NATIVE AMERICAN FEMALE':   {'democrat' = 0}

|        |        |        |        |        |        |        'HAWAIIAN PACIFIC FEMALE':   {'democrat' = 0}

|        |        |        |        |        'OTHER MISC':   {'democrat' = 0}

|        |        |        |        |        'CATHOLIC'
|        |        |        |        |        |        'ethnic_female'
|        |        |        |        |        |        |        'BLACK FEMALE':   {'democrat' = 0}

|   

1.2  Use the Gini coefficient.

In [29]:
catg_features = ['education', 'religion', 'ethnic_female', 'ethnic_male']
tot_n_row = df.shape[0] # total # of rows
id3_df = df
new_catg_features = catg_features[:]

def cal_gini(df, label): # df: dataframe, label: name of the label column of df
  n_row = len(df[label]) # number of rows
  n_0 = list(df[label].values == 0).count(True)
  n_1 = list(df[label].values == 1).count(True)
  gini = 1 - ((n_1/n_row) ** 2 + (n_0/n_row) ** 2)

  label_v_num = {0: n_0, 1: n_1} # contains label values and their corrsponding amount
  # gini_list = [gini, most represented value]
  gini_list = [gini, max(label_v_num, key=label_v_num.get)]
  return gini_list

def get_best_feature_gini(features, df): # features: column names list [str, str]
  features_gini_index = {} # contains every feature's IG
  for feature in features:
    gini_index = 0
    for sample in df[feature].unique(): # obtain distinct sample names in a feature to calculate its entropy
      pr = (list(df[feature].values == sample).count(True)) / tot_n_row
      # conditional entropy
      temp_df = df.loc[df[feature].values == sample]
      gini_index += pr * cal_gini(temp_df, 'democrat')[0]
    features_gini_index[feature] = gini_index
  return max(features_gini_index, key = features_gini_index.get)

def id3_gini(id3_df, catg_features):
  tree = Tree('democrat') # set a tree node with feature = democrat since this node could be a leaf representing democrat value. If it's not a leaf, we reset the feature name in the 'else' branch.
  gini_list = cal_gini(id3_df, 'democrat')
  
  if gini_list[0] == 0: # check if node is pure
    tree.set_label(gini_list[1])
    return tree
  elif len(catg_features) == 0: # check if categorical features is empty
    tree.set_label(gini_list[1])
    return tree
  else:
    # obtain best feature and add it to a new node
    best_feature = get_best_feature_gini(catg_features, id3_df)
    tree.set_feature(best_feature)
    # remove the best feature
    catg_features.remove(best_feature)
    i = 0 # index of child node
    for value in list(id3_df[best_feature].unique()):
      # create child node for tree with best feature
      node = Tree(value)
      tree.add_child(node)
      value_df = id3_df.loc[id3_df[best_feature].values == value] # create a dataframe only contain 'value'
      if value_df.shape[0] == 0: # value_df is empty
        leaf = Tree('democrat')
        gini_list2 = cal_gini(value_df, 'democrat')
        leaf.set_label(gini_list2[1])
        tree.children[i].add_child(leaf)
      else:
        new_catg_features = catg_features[:] # obtain the copy of catg_features so that the iteration won't damage the orginal catg_features
        tree.children[i].add_child(id3_gini(value_df, new_catg_features))
      i += 1
  return tree
id3_tree = id3_gini(df, new_catg_features)
# print the decision tree!
str(id3_tree)
print(id3_tree)


'religion'
|        'CHRISTIAN GENERIC'
|        |        'ethnic_female'
|        |        |        'NATIVE AMERICAN FEMALE'
|        |        |        |        'ethnic_male'
|        |        |        |        |        'MULTI MALE'
|        |        |        |        |        |        'education'
|        |        |        |        |        |        |        'Bachelor':   {'democrat' = 0}

|        |        |        |        |        |        |        'HighSchool':   {'democrat' = 1}

|        |        |        |        |        |        |        'College':   {'democrat' = 1}

|        |        |        |        |        'NATIVE AMERICAN MALE'
|        |        |        |        |        |        'education'
|        |        |        |        |        |        |        'Bachelor':   {'democrat' = 0}

|        |        |        |        |        |        |        'LessThenHighSchool':   {'democrat' = 0}

|        |        |        |        |        |        |        'HighSchool':   

Summary:

1. During implementing functions for Entropy and Gini coefficient, I found computing Gini Index is faster than computing Information Gain.

2. Tree generated by using Entropy is better than using Gini coefficient since it has less branches than Gini.

1.3 Evaluation

I am sorry I didn't do Evaluation part because I ran out of time.